In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import json
import os

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if device == "cpu":
    print("⚠️  WARNING: Training on CPU will be VERY slow!")
    print("   Consider using Google Colab with free GPU")
    print("   Continuing anyway...")

# Create output directory
os.makedirs('models', exist_ok=True)
os.makedirs('results/training', exist_ok=True)

In [ ]:
class ContentModerationDataset(Dataset):
    def __init__(self, dataframe, processor, max_length=77):
        self.df = dataframe.reset_index(drop=True)
        self.processor = processor
        self.max_length = max_length
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load image
        try:
            image = Image.open(row['image_path']).convert('RGB')
        except Exception as e:
            # Return a blank image on error
            image = Image.new('RGB', (224, 224), color='white')
        
        # Label: 0 = safe, 1 = unsafe
        label = 1 if row['label'] == 'unsafe' else 0
        
        # Process image (processor handles resizing, normalization)
        inputs = self.processor(
            images=image,
            return_tensors="pt",
            padding=True
        )
        
        # Remove batch dimension
        pixel_values = inputs['pixel_values'].squeeze(0)
        
        return {
            'pixel_values': pixel_values,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load data
df = pd.read_csv('datasets/metadata/combined_dataset.csv')

train_df = df[df['split'] == 'train'].copy()
val_df = df[df['split'] == 'val'].copy()
test_df = df[df['split'] == 'test'].copy()

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")
print(f"\nTrain label distribution:")
print(train_df['label'].value_counts())

In [ ]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Create classification head
class CLIPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=2):
        super().__init__()
        self.clip = clip_model
        self.classifier = nn.Linear(512, num_classes)  # CLIP base has 512-dim embeddings
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, pixel_values):
        # Get image features from CLIP
        outputs = self.clip.get_image_features(pixel_values=pixel_values)
        # Apply dropout
        outputs = self.dropout(outputs)
        # Classify
        logits = self.classifier(outputs)
        return logits

model = CLIPClassifier(clip_model)
model = model.to(device)

print(f"✅ Model initialized")
print(f"   Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"   Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Create datasets
train_dataset = ContentModerationDataset(train_df, processor)
val_dataset = ContentModerationDataset(val_df, processor)
test_dataset = ContentModerationDataset(test_df, processor)

print(f"\n✅ Datasets created")

In [ ]:
# Training hyperparameters
BATCH_SIZE = 32  # Reduce if out of memory
EPOCHS = 5
LEARNING_RATE = 1e-5  # Small LR for fine-tuning
WEIGHT_DECAY = 0.01

# Create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2
)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

print("Training Configuration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Total training batches: {len(train_loader)}")
print(f"  Total validation batches: {len(val_loader)}")

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc="Training")
    for batch in pbar:
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(pixel_values)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{100.*correct/total:.2f}%'})
    
    return total_loss / len(loader), 100. * correct / total

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            pixel_values = batch['pixel_values'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(pixel_values)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

best_val_acc = 0
best_model_path = 'models/clip_finetuned_best.pt'

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print("-" * 60)
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"\nEpoch {epoch+1} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
        }, best_model_path)
        print(f"  ✅ New best model saved! Val Acc: {val_acc:.2f}%")

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)
print(f"Best Validation Accuracy: {best_val_acc:.2f}%")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

epochs_range = range(1, EPOCHS + 1)

# Loss plot
ax1 = axes[0]
ax1.plot(epochs_range, history['train_loss'], 'b-', label='Train Loss')
ax1.plot(epochs_range, history['val_loss'], 'r-', label='Val Loss')
ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(alpha=0.3)

# Accuracy plot
ax2 = axes[1]
ax2.plot(epochs_range, history['train_acc'], 'b-', label='Train Acc')
ax2.plot(epochs_range, history['val_acc'], 'r-', label='Val Acc')
ax2.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('results/training/training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Saved training history plot")

# Save history
with open('results/training/history.json', 'w') as f:
    json.dump(history, f, indent=2)

In [ ]:
# Load best model
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("Evaluating on test set...")

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(pixel_values)
        probs = torch.softmax(outputs, dim=1)
        _, predicted = outputs.max(1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

# Convert to numpy
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_probs = np.array(all_probs)

# Calculate metrics
test_accuracy = accuracy_score(all_labels, all_preds)

print("\n" + "="*60)
print("FINE-TUNED MODEL - TEST SET RESULTS")
print("="*60)
print(f"\nTest Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Classification report
target_names = ['Safe', 'Unsafe']
print(f"\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=target_names))

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print(f"\nConfusion Matrix:")
print(f"                Predicted")
print(f"              Safe  Unsafe")
print(f"Actual Safe   {cm[0][0]:4d}  {cm[0][1]:4d}")
print(f"       Unsafe {cm[1][0]:4d}  {cm[1][1]:4d}")

In [ ]:
# Load baseline results
with open('results/clip_baseline_metrics.json', 'r') as f:
    baseline_results = json.load(f)

print("\n" + "="*60)
print("BASELINE vs FINE-TUNED COMPARISON")
print("="*60)
print(f"\nBaseline (Zero-Shot):")
print(f"  Accuracy: {baseline_results['accuracy']*100:.2f}%")
print(f"  F1 Score: {baseline_results['f1_score']:.4f}")

print(f"\nFine-Tuned:")
print(f"  Accuracy: {test_accuracy*100:.2f}%")

improvement = (test_accuracy - baseline_results['accuracy']) * 100
print(f"\n📈 Improvement: +{improvement:.2f} percentage points")
print(f"   ({improvement/baseline_results['accuracy']*100:.1f}% relative improvement)")

# Save fine-tuned results
finetuned_results = {
    'model': 'CLIP Fine-Tuned',
    'test_accuracy': float(test_accuracy),
    'best_val_accuracy': float(best_val_acc),
    'epochs_trained': EPOCHS,
    'baseline_accuracy': baseline_results['accuracy'],
    'improvement': float(improvement),
}

with open('results/clip_finetuned_metrics.json', 'w') as f:
    json.dump(finetuned_results, f, indent=2)

print("\n✅ Results saved to results/clip_finetuned_metrics.json")
print("✅ Model saved to models/clip_finetuned_best.pt")